In [13]:
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import pandas as pd
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
import MetaTrader5 as mt5
import pytz

#https://www.mql5.com/en/docs/python_metatrader5

In [23]:
#conversão pro metatrader5:
def adicionar_minutos(data, num_candles, minuto_candle):
    minutos = num_candles * minuto_candle
    data_modificada = data + timedelta(minutes=minutos)
    return data_modificada

class FinancialData:
    global timezone
    timezone = pytz.timezone("Etc/UTC")

    def tabela_de_precos_inicial():
        global num_candles,ano,dia,mes,simbolo
        num_candles = int(input("Quantos candles você quer?"))
        print("Os dados a seguir serão coletadosem ordem reversa, ou seja, da data que você colocou - numero de candles")
        ano = int(input("Coloque o ano: "))
        mes = int(input("Coloque o mes: "))
        dia = int(input("Coloque o dia: "))
        simbolo = str(input("Coloque o simbolo em letras maiusculas"))

        data_final = datetime(ano,mes,dia, tzinfo=timezone) 

        # establish connection to MetaTrader 5 terminal
        if not mt5.initialize():
            print("initialize() failed, error code =",mt5.last_error())
            quit()

        rates = mt5.copy_rates_from(simbolo, mt5.TIMEFRAME_M15, data_final, num_candles)

        # shut down connection to the MetaTrader 5 terminal
        mt5.shutdown()

        # create DataFrame out of the obtained data
        rates_frame = pd.DataFrame(rates)
        # convert time in seconds into the 'datetime' format
        rates_frame['time']=pd.to_datetime(rates_frame['time'], unit='s')
        rates_frame.drop(['tick_volume', 'spread', 'real_volume'], axis=1, inplace=True)

        return rates_frame, data_final

    def tabela_de_precos(data_final):
        global num_candles,ano,dia,mes,simbolo
        data_final = adicionar_minutos(data_final,1,15)

        # establish connection to MetaTrader 5 terminal
        if not mt5.initialize():
            print("initialize() failed, error code =",mt5.last_error())
            quit()

        rates = mt5.copy_rates_from(simbolo, mt5.TIMEFRAME_M15, data_final, num_candles)

        # shut down connection to the MetaTrader 5 terminal
        mt5.shutdown()

        # create DataFrame out of the obtained data
        rates_frame = pd.DataFrame(rates)
                
        # convert time in seconds into the 'datetime' format
        rates_frame['time']=pd.to_datetime(rates_frame['time'], unit='s')
        rates_frame.drop(['tick_volume', 'spread', 'real_volume'], axis=1, inplace=True)

        return rates_frame, data_final

    def verificacao_data_de_hoje(data_final):
        if data_final <= datetime.now() - timedelta(hours=1):
            # Pare o loop ou execute a lógica desejada
            pass

In [24]:
resultado = FinancialData.tabela_de_precos_inicial()
print("Resultado 1:")
print(resultado)
resultado2 = FinancialData.tabela_de_precos(resultado[1])
print("Resultado 2:")
print(resultado2)

Os dados a seguir serão coletadosem ordem reversa, ou seja, da data que você colocou - numero de candles
Resultado 1:
(                  time     open     high      low    close
0  2020-08-18 23:15:00  1.19353  1.19372  1.19344  1.19363
1  2020-08-18 23:30:00  1.19363  1.19377  1.19342  1.19348
2  2020-08-18 23:45:00  1.19349  1.19349  1.19289  1.19300
3  2020-08-19 00:00:00  1.19300  1.19308  1.19287  1.19303
4  2020-08-19 00:15:00  1.19303  1.19333  1.19303  1.19324
..                 ...      ...      ...      ...      ...
95 2020-08-19 23:00:00  1.18453  1.18467  1.18419  1.18438
96 2020-08-19 23:15:00  1.18435  1.18471  1.18423  1.18424
97 2020-08-19 23:30:00  1.18424  1.18424  1.18364  1.18383
98 2020-08-19 23:45:00  1.18381  1.18427  1.18362  1.18363
99 2020-08-20 00:00:00  1.18371  1.18404  1.18366  1.18404

[100 rows x 5 columns], datetime.datetime(2020, 8, 20, 0, 0, tzinfo=<StaticTzInfo 'Etc/UTC'>))
Resultado 2:
(                  time     open     high      low    close
0  2